# Informe Modelo Predictivo Popularidad de Canciones en Spotify
<p style="text-align: justify;">
Este informe presenta el desarrollo de un modelo de aprendizaje automático, cuyo objetivo es predecir el nivel de popularidad de canciones en Spotify. A lo largo del documento se describen las etapas fundamentales del proceso, incluyendo el preprocesamiento de datos, la selección y calibración del modelo, el entrenamiento y evaluación del rendimiento del mismo. Finalmente, se presenta el procedimiento de disponibilización del modelo predictivo mediante una API.<p>

In [1]:
## Librerias a Importar
import warnings
warnings.filterwarnings('ignore')
# Manipulación de datos
import pandas as pd
import numpy as np
# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Modelos base
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    BaggingRegressor,
    StackingRegressor
)
from sklearn.linear_model import (
    ElasticNetCV, 
    RidgeCV, 
    LassoCV, 
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
# Preprocesamiento y selección de características
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectFromModel

## Preprocesamiento de Datos
<p style="text-align: justify;">
En este proyecto se usó un conjunto de datos de popularidad en canciones, donde cada observación representa una canción y se tienen variables como: duración de la canción, acusticidad y tiempo, entre otras. El objetivo del modelo que se presentará más adelante es predecir qué tan popular es la canción.<p>

In [2]:
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTrain_Spotify.csv')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTest_Spotify.csv', index_col=0)

In [3]:
print(tabulate(dataTraining.head(), headers='keys', tablefmt='psql'))

+----+--------------+------------------------+-------------------+----------------------+----------------------------------------+---------------+------------+----------------+----------+-------+------------+--------+---------------+----------------+--------------------+------------+-----------+---------+------------------+---------------+--------------+
|    |   Unnamed: 0 | track_id               | artists           | album_name           | track_name                             |   duration_ms | explicit   |   danceability |   energy |   key |   loudness |   mode |   speechiness |   acousticness |   instrumentalness |   liveness |   valence |   tempo |   time_signature | track_genre   |   popularity |
|----+--------------+------------------------+-------------------+----------------------+----------------------------------------+---------------+------------+----------------+----------+-------+------------+--------+---------------+----------------+--------------------+------------+--

<p style="text-align: justify;">
A continuación, se valida la dimensión de los datos, los tipos de variables y se indentifica la existencia de valores faltantes y duplicados. Según los resultados, se identificaron valores duplicados y algunas variables string y categoricas. De igual forma se identifica que existen variables tipo <code>int64</code> y <code>float64</code>, los cuales podrían generan un uso importante de la memoria.<p>

In [4]:
# Revisión inicial de datos
print("Dimensión de los datos de entrenamiento:", dataTraining.shape)

if dataTraining.isnull().sum().sum() > 0: # Verifica si hay valores nulos
    print("Valores nulos en los datos de entrenamiento:")
    print(dataTraining.isnull().sum())
else:
    print("No hay valores nulos en los datos de entrenamiento.")

print("Número de canciones duplicadas según Track_ID:")
print(dataTraining['track_id'].duplicated().sum())

print("Tipos de variables en la base de datos:")
print(dataTraining.dtypes.unique())

print("Variables string y categóricas en la base de datos:")
print(dataTraining.select_dtypes(include=['object', 'bool']).info())

Dimensión de los datos de entrenamiento: (79800, 21)
No hay valores nulos en los datos de entrenamiento.
Número de canciones duplicadas según Track_ID:
13080
Tipos de variables en la base de datos:
[dtype('int64') dtype('O') dtype('bool') dtype('float64')]
Variables string y categóricas en la base de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79800 entries, 0 to 79799
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_id     79800 non-null  object
 1   artists      79800 non-null  object
 2   album_name   79800 non-null  object
 3   track_name   79800 non-null  object
 4   explicit     79800 non-null  bool  
 5   track_genre  79800 non-null  object
dtypes: bool(1), object(5)
memory usage: 3.1+ MB
None


<p style="text-align: justify;">
A continuación, se realizarán algunos ajustes a la base de datos. En primer lugar, se elimnan las observaciones duplicadas según la variable <code>track_id</code>, porteriormente se optimiza el uso de memoria del dataset <code>dataTraining</code> ajustando los tipos de datos según su contenido. Este último procedimiento es relevante porque reduce significativamente el consumo de memoria, mejorando la eficiencia del procesamiento y el rendimiento del modelo. 

In [5]:
# Eliminación de duplicados
dataTraining = dataTraining.drop_duplicates(subset='track_id', keep='first')

In [6]:
# Transformación tipológica de variables con el objetivo de reducir el uso de memoria
# Se recomienda usar float32 en lugar de float64 y int32 en lugar de int64
for col in dataTraining.columns:
    col_type = dataTraining[col].dtype

    if col_type == 'float64':
        dataTraining[col] = dataTraining[col].astype('float32')
    elif col_type == 'int64':
        dataTraining[col] = dataTraining[col].astype('int32')
    elif col_type == 'bool':
        dataTraining[col] = dataTraining[col].astype('int8')

<p style="text-align: justify;">
Posteriormente, se eliminaron columnas irrelevantes, como <code>Unnamed: 0</code>, presentes en la base de datos y se crean nuevas variables derivadas de las características originales, tales como la longitud del nombre de la canción, la densidad de tempo, la interacción entre energía y bailabilidad, y una variable binaria de acusticidad. Finalmente, se realizó una selección de variables, eliminando columnas como <code>track_id</code>, <code>track_name</code>, <code>artists</code>, <code>album_name</code> y <code>track_genre</code>, para conformar el conjunto de características que serían utilizadas en el entrenamiento del modelo.<p>

In [7]:
# Eliminación de variables string
for col in ['Unnamed: 0']:
    if col in dataTraining.columns: dataTraining.drop(columns=col, inplace=True)
    if col in dataTesting.columns: dataTesting.drop(columns=col, inplace=True)

In [8]:
# Creación de variables nuevas
# Se crean variables que pueden ser útiles para el modelo
for df in [dataTraining, dataTesting]:
    df['track_name_length'] = df['track_name'].apply(lambda x: len(str(x)))
    df['tempo_density'] = df['tempo'] / df['duration_ms']
    df['energy_danceability'] = df['energy'] * df['danceability']
    df['acousticness_bin'] = (df['acousticness'] > 0.5).astype(int)

In [9]:
#  Selección de columnas y escalado
drop_cols = ['track_id', 'track_name', 'artists', 'album_name', 'track_genre']
features = dataTraining.drop(columns=drop_cols + ['popularity']).columns.tolist()


Para asegurar que no hubiera errores durante el entrenamiento, se reemplazan valores infinitos o faltantes usando la mediana:


In [10]:
# Reemplazar valores infinitos por NaN y luego llenar NaN con la mediana
for df in [dataTraining, dataTesting]:
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.fillna(df.median(numeric_only=True), inplace=True)

<p style="text-align: justify;">
Los datos fueron escalados utilizando <code>RobustScaler</code>. El ajuste del escalador se realizó únicamente sobre el conjunto de entrenamiento mediante el método <code>.fit_transform()</code>, y posteriormente se aplicó la transformación al conjunto de prueba utilizando <code>.transform()</code>. Esto asegura que el modelo solo utilice información disponible durante el entrenamiento y que las evaluaciones en el conjunto de prueba sean realistas..<p>


In [11]:
# Escalado de datos
scaler = RobustScaler()
X = scaler.fit_transform(dataTraining[features])
XTesting = scaler.transform(dataTesting[features])
y = dataTraining['popularity']

<p style="text-align: justify;">
Finalmente, como parte del preprocesamiento de datos, se realizó una selección de variables utilizando un modelo de Extreme Gradient Boosting (XGB) como base. Esta técnica permitió reducir la dimensionalidad del conjunto de datos y mejorar la eficiencia del modelo sin afectar su capacidad predictiva. Como resultado, se seleccionaron 7 variables: 4 provenientes del conjunto de datos original y 3 generadas durante las etapas anteriores de transformación.<p>

In [12]:
# Selección de características con XGBRegressor
selector = SelectFromModel(XGBRegressor(n_estimators=100, random_state=42))
selector.fit(X, y)
X_sel = selector.transform(X)
X_test_sel = selector.transform(XTesting)

In [14]:
# Variables predictoras
selected_columns = dataTraining[features].columns[selector.get_support()].tolist()
print(tabulate([[col] for col in selected_columns], headers=["Columnas Seleccionadas"], tablefmt="psql"))

+--------------------------+
| Columnas Seleccionadas   |
|--------------------------|
| explicit                 |
| acousticness             |
| instrumentalness         |
| valence                  |
| time_signature           |
| track_name_length        |
| energy_danceability      |
+--------------------------+


## Separación de Datos en Entrenamiento y Prueba
<p style="text-align: justify;">
Se dividió el conjunto de datos en dos subconjuntos: uno de entrenamiento y otro de validación, utilizando una proporción del 80 % para entrenamiento y 20 % para validación.<p>

In [17]:
# Dividir los datos en entrenamiento y evaluación
X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=0.2, random_state=42)

## Selección y Calibración del Modelo

#### (a) Selección del Modelo Predictivo y Justificación
<p style="text-align: justify;">
A través de un proceso iterativo de prueba y error con diferentes modelos predictivos, se seleccionó el modelo de Stacking como el modelo definitivo para participar en la competencia de predicción del nivel de popularidad de las canciones en Spotify.Para la construcción del Stacking, se eligieron diversos modelos base con arquitecturas distintas, con el objetivo de aportar diversidad y minimizar el riesgo de que todos cometieran los mismos errores. Entre los modelos seleccionados se encuentran:<p>

- <code>SVR</code> (Support Vector Regressor) con núcleo rbf, ideal para capturar relaciones no lineales complejas.
- Ensambles basados en árboles como <code>RandomForestRegressor</code>, <code>GradientBoostingRegressor</code>, <code>ExtraTreesRegressor</code> y <code>BaggingRegressor</code>, conocidos por su robustez frente al sobreajuste y su buena capacidad de generalización.
- Modelos de boosting como <code>XGBoost</code>, <code>LightGBM</code> y <code>CatBoost</code>, altamente eficientes en tareas de predicción.
- Modelos lineales (<code>ElasticNetCV</code>, <code>RidgeCV</code>, <code>LassoCV</code>), útiles para capturar relaciones lineales y aplicar regularización que ayuda a evitar el sobreajuste.
- <code>KNeighborsRegressor</code>, que resulta efectivo para detectar patrones locales en los datos.

<p style="text-align: justify;">
Inicialmente, se intentó utilizar <code>GridSearchCV</code> para optimizar los hiperparámetros de cada modelo base. Sin embargo, este enfoque resultó ser demasiado costoso en tiempo y recursos computacionales, y no ofreció mejoras significativas en el desempeño respecto a configuraciones manuales basadas en experiencia previa e iteraciones manuales sobre el conjunto de datos.<p>
<p style="text-align: justify;">
Por este motivo, se optó por emplear configuraciones predefinidas que demostraron ser eficaces, permitiendo un entrenamiento más eficiente sin comprometer el rendimiento final del modelo.<p>
<p style="text-align: justify;">
A continuación, se presenta el procedimiento de calibración, entrenamiento, predicción y evaluación del desempeño del modelo seleccionado.<p>

In [ ]:
# Definir los modelos base
base_models = [
    ('svr', SVR(kernel='rbf', C=10, epsilon=0.2)),
    ('rf', RandomForestRegressor(n_estimators=300, max_depth=30, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=300, max_depth=10, random_state=42)),
    ('et', ExtraTreesRegressor(n_estimators=300, max_depth=30, random_state=42)),
    ('bag', BaggingRegressor(n_estimators=300, max_samples=0.8, max_features=0.8, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=300, learning_rate=0.075, max_depth=10, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=300, learning_rate=0.075, max_depth=10, random_state=42)),
    ('catboost', CatBoostRegressor(iterations=300, depth=10, learning_rate=0.075, random_seed=42, verbose=False)),
    ('elasticnet', ElasticNetCV(cv=5)),
    ('ridge', RidgeCV()),
    ('lasso', LassoCV()),
    ('knn', KNeighborsRegressor(n_neighbors=10))
]

### **(b) Parámetros usados en los modelos base**
Para construir el modelo de `Stacking`, se usaron varios modelos base con configuraciones diferentes. Algunos modelos comparten parámetros similares, así que aquí te dejo un resumen agrupado y explicado de forma sencilla sobre lo que hace cada uno.

#### **Modelos basados en árboles (RandomForest, ExtraTrees, Bagging)**

```python
n_estimators=300, max_depth=30

n_estimators: Es la cantidad de árboles que se entrenan. Al usar 300, el modelo se vuelve más estable y menos sensible al ruido.

max_depth: Es la profundidad máxima de los árboles. Mientras más grande, más complejo el modelo (pero también puede sobreajustar si es muy alto).


#### Para **Bagging**, también se usa:

max_samples=0.8, max_features=0.8

Esto hace que cada árbol vea solo una parte de los datos y de las variables, lo que genera más variedad entre ellos y mejora el desempeño general.

#### Modelos de boosting (XGBoost, LightGBM, CatBoost, GradientBoosting)

n_estimators=300, learning_rate=0.075, max_depth=10

Estos modelos aprenden de los errores que cometen paso a paso.

learning_rate: Define qué tanto aprende el modelo en cada paso. Un valor bajo (como 0.075) ayuda a que el modelo generalice mejor y no se sobreentrene.

max_depth o depth: Controla qué tan profundo puede ser cada árbol. Se usa 10 para permitir algo de complejidad sin exagerar.

#### **Modelos lineales (RidgeCV, LassoCV, ElasticNetCV)**
No necesitan que les pongamos parámetros manualmente porque ellos mismos los calculan por dentro con validación cruzada.

Son útiles porque ayudan a mantener el modelo balanceado, aportando relaciones lineales y evitando que todo dependa de modelos más complejos.

#### **SVR (Support Vector Regressor)**

kernel='rbf', C=10, epsilon=0.2

Este modelo es útil para relaciones no lineales.

C=10 hace que el modelo trate de ajustar más a los datos (menos tolerancia al error).

epsilon=0.2 permite que algunos errores pequeños no se penalicen, lo que lo hace un poco más flexible.

#### **KNeighborsRegressor**

n_neighbors=10

Este modelo se basa en los 10 puntos más parecidos al que se quiere predecir.

Al usar más vecinos, las predicciones se suavizan, lo cual puede ayudar cuando hay ruido en los datos.

## Entrenamiento del Modelo

In [ ]:
# Ensamblaje de modelos

# Estrategia de validación cruzada
cv_strategy = KFold(n_splits=15, shuffle=True, random_state=42)

# Crear el modelo de apilamiento
stacking_model = StackingRegressor(
    estimators=base_models,
    final_estimator=XGBRegressor(n_estimators=200, learning_rate=0.075, max_depth=10, random_state=42),
    passthrough=True,
    n_jobs=-1,
    cv=cv_strategy

Usamos un `StackingRegressor` que combina varios modelos diferentes como base, y al final un `XGBRegressor` como meta-modelo. Esto nos permite aprovechar lo mejor de cada modelo y mejorar la precisión general.

La validación cruzada se hizo con `KFold` (15 divisiones) para evaluar el desempeño de forma más confiable. Además, activamos `passthrough=True` para que el meta-modelo pueda usar tanto las predicciones de los modelos base como las variables originales, lo que mejora su capacidad de aprender.

Elegimos XGBoost como meta-modelo porque es muy bueno captando relaciones complejas, y funciona bien en tareas de regresión como esta.

In [ ]:
# Entrenar el modelo de stacking con los datos transformados
stacking_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de validación
y_pred = stacking_model.predict(X_test)

# Calcular la raíz del error cuadrático medio (RMSE)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE validación local: {rmse:.5f}")

La métrica seleccionada (RMSE) es apropiada para tareas de regresión y penaliza fuertemente los errores grandes. Se calcula en el conjunto de validación, asegurando que el resultado refleje desempeño fuera de muestra.

Primero se usó train_test_split para hacer una validación rápida del modelo y ver qué tan bien predecía en una parte de los datos. Sin embargo, la evaluación más importante se hizo con validación cruzada usando KFold, porque permite probar el modelo varias veces con diferentes divisiones, lo cual da una idea más confiable de su rendimiento.

Luego, antes de hacer la predicción final sobre los datos de prueba (X_test_sel), se volvió a entrenar el modelo usando todo el conjunto de entrenamiento disponible (X_sel y y), para aprovechar al máximo la información y asegurar mejores resultados.

#### **Evaluación del Desempeño**

In [ ]:
# Entrenar el modelo de stacking con los datos transformados
stacking_model.fit(X_sel, y)

# Generar predicciones finales en el conjunto de prueba
test_pred = stacking_model.predict(X_test_sel)

# Evaluación del modelo
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {RMSE:.5f}")

In [ ]:
# Creaciión del dataframe de envío
submission = pd.DataFrame({'ID': dataTesting.index, 'popularity': test_pred})

In [ ]:
# Crear el archivo de envío para Kaggle
submission.to_csv('test_submission_file.csv', index=False)
submission.head()

Predicción final y exportación
Se generan predicciones sobre el conjunto de test utilizando el modelo ensamblado. Finalmente, los resultados se guardan en un archivo CSV conforme al formato de envío de Kaggle.

## Disponibilización del Modelo

El proceso para asegurar la disponibilidad del modelo se estructuró en tres pasos principales:

#### Construcción y almacenamiento del modelo
Se construyó un modelo y se almacenó en un archivo .pkl.
Debido a las limitaciones de tamaño en GitHub, no se utilizó el modelo de la competencia. En su lugar, se implementó un modelo de Random Forest con 50 muestras, permitiendo que el archivo pudiera ser subido a la plataforma.

#### Creación de la función en un archivo .py
El archivo .pkl generado fue utilizado para crear una función dentro de un archivo .py.
Esta función fue diseñada para ser importada posteriormente, facilitando la modularidad y reutilización del código.

#### Desarrollo de la API
Finalmente, se construyó la API en otro archivo .py.
Esta API importa la función previamente creada y configura un método GET, el cual recibe los parámetros necesarios para ejecutar la predicción del modelo de forma sencilla y estructurada.

#### Servidor utilizado

<img src="Imagenes_Disponibilidad/Dispo_1.png">
<img src="Imagenes_Disponibilidad/Dispo_2.png">

API: http://54.242.6.90:5000

## Resultados de la API

In [6]:
dataTesting.head(2)

,track_id,artists,album_name,track_name,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,6KwkVtXm8OUp2XffN5k7lY,Hillsong Worship,No Other Name,No Other Name,440247,False,0.369,0.598,7,-6.984,1,0.0304,0.00511,0.000,0.176,0.0466,148.014,4,world-music
1,2dp5I5MJ8bQQHDoFaNRFtX,Internal Rot,Grieving Birth,Failed Organum,93933,False,0.171,0.997,7,-3.586,1,0.1180,0.00521,0.801,0.420,0.0294,122.223,4,grindcore


#### Entrada 0:

<img src="Imagenes_Disponibilidad/Dispo_3.png">

#### Entrada 1:

<img src="Imagenes_Disponibilidad/Dispo_4.png">


## Conclusiones